---

# Process the Performance Report

---

In [60]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys

In [61]:
start_time = time.time()

In [62]:
# Path to performance report
report_path = 'reports/'
performance_report              = report_path + 'performance_report.pkl'
performance_report_text_file    = report_path + 'performance_report.txt'

---

## 1. Read performance report

---

In [63]:
if os.path.exists(performance_report):
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
    print(f"The file {performance_report} exists and the Performance Report was read successfully")
else:
    print(f"******************************************************")
    print(f"The file: '{performance_report}' DOES NOT EXIST")
    print(f"******************************************************")

The file reports/performance_report.pkl exists and the Performance Report was read successfully


---

## 2. Write the Performance Report

---

In [64]:

perf_datasets_in = perf_report.keys()

perf_datasets_in

dict_keys(['2.0 StandardScaler Dataset', '2.1 MinMaxScaler Dataset', '3 Binary Dataset', '4 RandomUnderSampler Dataset', '5 RandomOverSampler Dataset', '7 SMOTE Dataset', '8 SMOTEENN Dataset'])

#### 2.1 Modify the order of the reported datasets

---

In [65]:
# Use this cell to re-order the datasets for the report

perf_datasets = perf_datasets_in

#### 2.2 Print the report

---

In [66]:
def print_performance_report(perf_report):
    # Print header and then every performance report in the reports dictionary

    print(f"**********************************************************")
    print(f"Diabetes Predictions Performance Summary Report")
    print(f"**********************************************************")

    print(f"----------------------------------------")
    print(f"Report Summary")
    print(f"----------------------------------------")

    for report in perf_report:
        print(f"Performance for: {report}")

    print(f"----------------------------------------")

    for report in perf_report:
        report_dict = perf_report[report]
        dataset_size = report_dict['dataset_size']
        report_df = report_dict['report']
        print(f"----------------------------------------")
        print(f"Performance for: {report}")
        print(f"Dataset Size:    {dataset_size[0]} Rows, {dataset_size[1]} Columns")
        print(f"----------------------------------------")
        print(f"{report_df.to_string(index=False)}")

In [67]:
# Generate the Performance Report and send prints to osc.stdout
with mlanlys.OutStreamCapture() as osc:
    print_performance_report(perf_report)

In [68]:
# osc.stdout contains the details of the performance report
# write the performance report to the performance_report_text_file
with open(performance_report_text_file, "w") as file:
    file.write(osc.stdout)

In [69]:
# Display the performance report here:
print(osc.stdout)

**********************************************************
Diabetes Predictions Performance Summary Report
**********************************************************
----------------------------------------
Report Summary
----------------------------------------
Performance for: 2.0 StandardScaler Dataset
Performance for: 2.1 MinMaxScaler Dataset
Performance for: 3 Binary Dataset
Performance for: 4 RandomUnderSampler Dataset
Performance for: 5 RandomOverSampler Dataset
Performance for: 7 SMOTE Dataset
Performance for: 8 SMOTEENN Dataset
----------------------------------------
----------------------------------------
Performance for: 2.0 StandardScaler Dataset
Dataset Size:    253680 Rows, 22 Columns
----------------------------------------
                new_column                  model slice  score  balanced_accuracy  roc_auc_score  Mean Squared Error  Accuracy  Precision  Recall  F1-score  Specificity  False Positive Rate  Matthews Correlation Coefficient
2.0 StandardScaler Datase

In [70]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 0.52 seconds:


## 3. Merge Performance Metrics into a single dataframe

In [71]:
# Concatenate all the Reports into one dataframe
df = pd.DataFrame()

for report in perf_report:
    report_dict = perf_report[report]
    report_df = report_dict['report']
    df = pd.concat([df, report_df], ignore_index=True)

In [72]:
df.rename(columns={ 'new_column': 'dataset'}, inplace=True)
df

,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,2.0 StandardScaler Dataset,DecisionTreeClassifier,Train,0.9936,0.9699,0.9999,0.0222,0.9946,0.9991,0.9621,0.9803,0.9999,0.0001,NaN
1,2.0 StandardScaler Dataset,DecisionTreeClassifier,Test,0.7697,0.4087,0.5724,0.8013,0.7994,0.3018,0.3269,0.3138,0.8766,0.1234,0.1968
2,2.0 StandardScaler Dataset,LogisticRegression,Train,0.8465,0.3841,0.7820,0.5582,0.8634,0.5262,0.1764,0.2642,0.9744,0.0256,0.2475
3,2.0 StandardScaler Dataset,LogisticRegression,Test,0.8461,0.3848,0.7808,0.5634,0.8620,0.5237,0.1798,0.2677,0.9733,0.0267,0.2483
4,2.1 MinMaxScaler Dataset,DecisionTreeClassifier,Train,0.9935,0.9674,0.9999,0.0222,0.9946,0.9986,0.9628,0.9804,0.9998,0.0002,NaN
5,2.1 MinMaxScaler Dataset,DecisionTreeClassifier,Test,0.7680,0.4036,0.5683,0.8048,0.7992,0.2956,0.3257,0.3099,0.8753,0.1247,0.1931
6,2.1 MinMaxScaler Dataset,LogisticRegression,Train,0.8465,0.3847,0.7819,0.5594,0.8630,0.5283,0.1786,0.2670,0.9741,0.0259,0.2496
7,2.1 MinMaxScaler Dataset,LogisticRegression,Test,0.8462,0.3839,0.7804,0.5602,0.8630,0.5154,0.1774,0.2639,0.9732,0.0268,0.2442
8,3 Binary Dataset,DecisionTreeClassifier,Train,0.9946,0.9811,0.9999,0.0054,0.9946,0.9991,0.9623,0.9804,0.9999,0.0001,NaN
9,3 Binary Dataset,DecisionTreeClassifier,Test,0.7950,0.5963,0.5957,0.2050,0.7950,0.2876,0.3211,0.3034,0.8715,0.1285,0.1840


## 4. Filter and sort test results

In [73]:
# Sort Dataframe by column to maximize
maximize = 'Accuracy'
limits = {
    'balanced_accuracy': .5,
    'Precision': .2
}

# Create dataframe with only the test metrics
test_df = df[df['slice'] == 'Test']

limit_df = test_df.copy()

# limit the rows based on the values in the limits dictionary above
for limit in limits:
    print(limit)
    limit_value = limits[limit]
    limit_df = limit_df[limit_df[limit]>limit_value]

# sort by column_to_maximize fro highest to lowest
sorted_df = limit_df.sort_values(by=column_to_maximize, ascending=False, inplace=False)


sorted_df

balanced_accuracy
Precision


,dataset,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
11,3 Binary Dataset,LogisticRegression,Test,0.8629,0.5654,0.8196,0.1371,0.8629,0.5230,0.1533,0.2371,0.9774,0.0226,0.2288
27,8 SMOTEENN Dataset,LogisticRegression,Test,0.8623,0.5648,0.8237,0.1377,0.8623,0.5426,0.1504,0.2355,0.9792,0.0208,0.2327
17,5 RandomOverSampler Dataset,DecisionTreeClassifier,Test,0.7969,0.5880,0.5881,0.2031,0.7969,0.2852,0.2975,0.2912,0.8784,0.1216,0.1729
21,7 SMOTE Dataset,DecisionTreeClassifier,Test,0.7965,0.6025,0.6017,0.2035,0.7965,0.2981,0.3328,0.3145,0.8722,0.1278,0.1959
9,3 Binary Dataset,DecisionTreeClassifier,Test,0.7950,0.5963,0.5957,0.2050,0.7950,0.2876,0.3211,0.3034,0.8715,0.1285,0.1840
25,8 SMOTEENN Dataset,DecisionTreeClassifier,Test,0.7927,0.5960,0.5951,0.2073,0.7927,0.2890,0.3220,0.3046,0.8699,0.1301,0.1836
23,7 SMOTE Dataset,LogisticRegression,Test,0.7336,0.7435,0.8203,0.2664,0.7336,0.3137,0.7573,0.4436,0.7297,0.2703,0.3573
19,5 RandomOverSampler Dataset,LogisticRegression,Test,0.7332,0.7478,0.8248,0.2668,0.7332,0.3149,0.7680,0.4467,0.7275,0.2725,0.3627
15,4 RandomUnderSampler Dataset,LogisticRegression,Test,0.7301,0.7446,0.8218,0.2699,0.7301,0.3094,0.7648,0.4406,0.7244,0.2756,0.3564
13,4 RandomUnderSampler Dataset,DecisionTreeClassifier,Test,0.6591,0.6574,0.6572,0.3409,0.6591,0.2371,0.6549,0.3482,0.6598,0.3402,0.2239


In [74]:
sorted_df.columns

Index(['dataset', 'model', 'slice', 'score', 'balanced_accuracy',
       'roc_auc_score', 'Mean Squared Error', 'Accuracy', 'Precision',
       'Recall', 'F1-score', 'Specificity', 'False Positive Rate',
       'Matthews Correlation Coefficient'],
      dtype='object')